### **Importación de datos, librerías y definición de funciones**

In [16]:
import pandas as pd
import json

In [17]:
data = pd.read_csv("../data/train.csv")

# df = data

# para v2 cargar --> convertir todo esto en un for donde se van limpiando los datos segun llegan (las columnas ya se añaden limpias al dataframe final)
# varios = pd.read_csv("data/misc_v2.csv")
# device = pd.read_csv("data/device_v2.csv")
# geoNetwork = pd.read_csv("data/geoNetwork_v2.csv")
# totals = pd.read_csv("data/totals_v2.csv")
# trafficSource = pd.read_csv("data/trafficSource_v2.csv")
# hitsFiles = glob.glob("*_hists.csv")
# for fl in hitsFiles:
  # hits = pd.read_csv("data/fl.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
# df = data.sample(n=3000)
df = data
df.shape

(903653, 12)

In [19]:
df['device']

485047    {"browser": "Safari", "browserVersion": "not a...
412338    {"browser": "Chrome", "browserVersion": "not a...
819187    {"browser": "Chrome", "browserVersion": "not a...
233082    {"browser": "Safari", "browserVersion": "not a...
345057    {"browser": "Chrome", "browserVersion": "not a...
553277    {"browser": "Chrome", "browserVersion": "not a...
420449    {"browser": "Chrome", "browserVersion": "not a...
236714    {"browser": "Chrome", "browserVersion": "not a...
467346    {"browser": "Chrome", "browserVersion": "not a...
244680    {"browser": "Chrome", "browserVersion": "not a...
385588    {"browser": "Chrome", "browserVersion": "not a...
32445     {"browser": "Safari", "browserVersion": "not a...
221393    {"browser": "Chrome", "browserVersion": "not a...
614731    {"browser": "Safari", "browserVersion": "not a...
454884    {"browser": "Chrome", "browserVersion": "not a...
734387    {"browser": "Chrome", "browserVersion": "not a...
602454    {"browser": "Chrome", "browser

In [4]:
# recursive flatten function from https://towardsdatascience.com/flattening-json-objects-in-python-f5343c794b10
# Call example:
  # flattenCol = [flatten_json(json.loads(d)) for d in df['trafficSource']]
  # dataFrameOfFlattenCol = pd.DataFrame(flattenCol)
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

  
# Call example:
  # df = extract_day(df, 'month')
def extract_date_info(df, parametro):
    if parametro == "date":
        df["date"] = [d.date() for d in df["visitStartTime"]]
    if parametro == "time":
        df["time"] = [d.time() for d in df["visitStartTime"]]
    if parametro == "day":
        df["day"] = df["visitStartTime"].dt.day
    if parametro == "month":
        df['month'] = df["visitStartTime"].dt.month
    if parametro == "weekday":
        df['weekday'] = df["visitStartTime"].dt.weekday
    if parametro == "weekofyear": 
        df['weekofyear'] = df["visitStartTime"].dt.weekofyear
    return df

### Limpieza 


In [5]:
# De misc mantenemos:
  # fullVisitorId
  # visitStartTime
  # channelGrouping

df["visitStartTime"] = pd.to_datetime(df["visitStartTime"], unit='s')
df.drop(['date','sessionId', 'visitId', 'socialEngagementType'], axis=1, inplace=True)

In [6]:
# De device mantenemos:
  # browser
  # deviceCategory
  # operatingSystem

deviceFlatten = [flatten_json(json.loads(d)) for d in df['device']]
devi = pd.DataFrame(deviceFlatten)


# quest: language?
devi.drop(columns=['browserSize', 'browserVersion', 'flashVersion', 'language', 
    'mobileDeviceBranding', 'mobileDeviceInfo', 'mobileDeviceMarketingName',
    'mobileDeviceModel', 'mobileInputSelector', 'operatingSystemVersion',
    'screenColors','screenResolution'], inplace=True)
isMobile = {False : 0, True : 1}
browser = {'Chrome': 1, 'Safari': 2, 'Firefox': 3, 'Internet Explorer': 4, 'Edge': 5, 
    'Android Webview': 0, 'Safari (in-app)': 0, 'Opera Mini': 0, 'Opera': 0,
    'UC Browser': 0}
category = {'desktop': 1, 'mobile': 2, 'tablet': 3}
operatingSystem = {'Windows': 1, 'Macintosh': 2, 'Android': 3, 'iOS': 4,'Linux': 5,
    'Chrome OS': 6, '(not set)': 0, 'Windows Phone': 0, 'BlackBerry': 0, 'Samsung': 0}

#we map our data to its corresponding value
devi["isMobile"] = devi["isMobile"].map(isMobile)
devi["browser"] = devi["browser"].map(browser)
devi["deviceCategory"] = devi["deviceCategory"].map(category)
devi["operatingSystem"] = devi["operatingSystem"].map(operatingSystem)

devi.drop(columns=['isMobile'], inplace=True)
df.drop(['device'], axis=1, inplace=True)
df = df.join(devi)
# df = pd.concat([df, devi])

In [7]:
# De geoNetwork mantenemos:
  # coutry
  # region (que contiene ciudad si no está disponible)
  # city

geoNetworkFlatten = [flatten_json(json.loads(d)) for d in df['geoNetwork']]
geo = pd.DataFrame(geoNetworkFlatten)
geo.drop(['latitude','longitude','networkLocation','cityId','continent','metro'], axis=1, inplace=True)

# QUEST: check that it works in the same way
# geo['region'].apply(lambda x: geo['country'] if x == '(not set)' else x)
# geo['region'].apply(lambda x: geo['country'] if x == 'not available in demo dataset' else x)


def region_country(geo):
    if geo['region'] == '(not set)':
        return geo['country']
    elif geo['region'] == 'not available in demo dataset':
        return geo['country']
    else:
        return geo['region']

geo['region']=geo.apply(region_country,axis=1)

df.drop(['geoNetwork'], axis=1, inplace=True)
df = df.join(geo)
# df = pd.concat([df, geo])

In [8]:
# De totals mantenemos:
  # hits
  # transactionRevenue
  # newVisits

#Transform json
totals_df = df['totals'].apply(json.loads).apply(pd.Series)

#Remove visits column as it does not provide any information
totals_df = totals_df.drop(['visits'], axis=1)

#Change nan in "newVisits" for 0. Binary, 1 it's new visit, 0 it's not. 
totals_df['newVisits'] = totals_df['newVisits'].fillna(0)

#Change nan in "transactionRevenue" for 0. 
totals_df['transactionRevenue'] = totals_df['transactionRevenue'].fillna(0)

#Change nan in "bounces" for 0
totals_df['bounces'] = totals_df['bounces'].fillna(0)

#There are some nan in pageviews but it is not trivial to find a value we can change these nan by logically.
#The corrlation with hits is 0.984, so we can keep hits that does not have any nan value. Keeping both could be redundant.
totals_df = totals_df.drop(['pageviews'], axis=1)

#Not 100% proven yet, but i have a big feeling that bounces is mainly 1 when there is only 1 hit by defenition.
#This means that even though the correlation doesn't provide a big value beetween them two, bounces is not giving much 
#additional info. Meaning using it is not relevant.
totals_df = totals_df.drop(['bounces'], axis=1)

df.drop(['totals'], axis=1, inplace=True)
df = df.join(totals_df)
# df = pd.concat([df, totals_df])

In [9]:
# De trafficSource mantenemos:
  # adHasContent
  # adPosition
  # campaign
  # isTrueDirect
  # medium
  # source

trafficFlatten = [flatten_json(json.loads(d)) for d in df['trafficSource']]
traffic = pd.DataFrame(trafficFlatten)
traffic.drop(['adwordsClickInfo_criteriaParameters', 'adwordsClickInfo_adNetworkType',
    'adwordsClickInfo_gclId', 'adwordsClickInfo_isVideoAd', 'adwordsClickInfo_page',
    'keyword', 'referralPath', ], axis=1, inplace=True)
# TO DO check campaignCode

# adContent to bool
traffic['adContent'] = traffic['adContent'].map(lambda x: 0 if str(x) == 'NoneType' else 1)

# rename to a simpler name
traffic.rename(index=str, columns={"adwordsClickInfo_page": "adPosition", "adContent": "adwordsClickInfo_slot", "": "adPosition"})

# TO DO: traffic['source'] regex for the main domain (last whatevs.com), and drop less than 1000


# TO DO: traffc['medium']
#     merge (not set) and (none)
#     cpc -> Cost Per Click
#     affiliate -> Affiliate marketing is a type of performance-based marketing in which a business rewards one or more affiliates for each visitor or customer brought by the affiliate's own marketing efforts.
#     cpm -> Cost per impression
# entender qué implican estos valores. ¿cpc vienen de search y cpm de webs? ¿Se juntan? ¿Importan?


df.drop(['trafficSource'], axis=1, inplace=True)
df = df.join(traffic)
# df = pd.concat([df, traffic])

In [10]:
# hits (only in v2)

In [13]:
df.shape

(903653, 23)

In [11]:
df.head()

,channelGrouping,fullVisitorId,visitId,visitNumber,visitStartTime,browser,deviceCategory,operatingSystem,city,country,...,hits,newVisits,transactionRevenue,adContent,adwordsClickInfo_slot,campaign,campaignCode,isTrueDirect,medium,source
0,Organic Search,1131660440785968503,1472830385,1,2016-09-02 15:33:05,1.0,1,1.0,Izmir,Turkey,...,1,1,0,1,NaN,(not set),NaN,NaN,organic,google
1,Organic Search,377306020877927890,1472880147,1,2016-09-03 05:22:27,3.0,1,2.0,not available in demo dataset,Australia,...,1,1,0,1,NaN,(not set),NaN,NaN,organic,google
2,Organic Search,3895546263509774583,1472865386,1,2016-09-03 01:16:26,1.0,1,1.0,Madrid,Spain,...,1,1,0,1,NaN,(not set),NaN,NaN,organic,google
3,Organic Search,4763447161404445595,1472881213,1,2016-09-03 05:40:13,0.0,1,5.0,not available in demo dataset,Indonesia,...,1,1,0,1,NaN,(not set),NaN,NaN,organic,google
4,Organic Search,27294437909732085,1472822600,2,2016-09-02 13:23:20,1.0,2,3.0,not available in demo dataset,United Kingdom,...,1,0,0,1,NaN,(not set),NaN,True,organic,google


In [15]:
df.to_csv("../data/train_v1_cleaned.csv")

### **Transformación**

### ** Minería de datos **

In [12]:
traffic

,adContent,adwordsClickInfo_slot,campaign,campaignCode,isTrueDirect,medium,source
0,1,NaN,(not set),NaN,NaN,organic,google
1,1,NaN,(not set),NaN,NaN,organic,google
2,1,NaN,(not set),NaN,NaN,organic,google
3,1,NaN,(not set),NaN,NaN,organic,google
4,1,NaN,(not set),NaN,True,organic,google
5,1,NaN,(not set),NaN,NaN,organic,google
6,1,NaN,(not set),NaN,NaN,organic,google
7,1,NaN,(not set),NaN,NaN,organic,google
8,1,NaN,(not set),NaN,NaN,organic,google
9,1,NaN,(not set),NaN,NaN,organic,google


### ** Interpretación y evaluación **